In [12]:
from pdfs_para_txt import pdfs_para_txt

In [13]:
pdfs_para_txt("data/PDFs", "data/txts")

Convertido: A Cartomante (Machado de Assis).pdf → A Cartomante (Machado de Assis).txt
Convertido: Dom Casmurro (Machado de Assis).pdf → Dom Casmurro (Machado de Assis).txt
Convertido: O Cortiço (Aluisio de Azevedo).pdf → O Cortiço (Aluisio de Azevedo).txt
Convertido: O Mulato (Aluisio Azevedo).pdf → O Mulato (Aluisio Azevedo).txt


In [14]:
import nltk

# Especificar manualmente o caminho do nltk_data no ambiente virtual
nltk.data.path.append('./.venv/nltk_data')  # Ajuste esse caminho conforme necessário

# Agora tente baixar novamente os pacotes
nltk.download('punkt', download_dir='./.venv/nltk_data')
nltk.download('stopwords', download_dir='./.venv/nltk_data')
nltk.download('punkt_tab', download_dir='./.venv/nltk_data')


[nltk_data] Downloading package punkt to ./.venv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to ./.venv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to ./.venv/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import os
from funcoes_preprocessamento import (normalizar_texto, 
                                      remover_caracteres_especiais, 
                                      tokenizar_texto, lematizar_tokens, 
                                      verificar_e_baixar_recurso)

# Garantir que os recursos necessários estejam disponíveis
verificar_e_baixar_recurso('punkt')
verificar_e_baixar_recurso('stopwords')

caminho_txts = "data/txts"

# Ler o primeiro arquivo txt no caminho
caminho_saida_pasta = "data/txts_processados"
os.makedirs(caminho_saida_pasta, exist_ok=True)

for arquivo_nome in os.listdir(caminho_txts):
    caminho_arquivo = os.path.join(caminho_txts, arquivo_nome)
    if os.path.isfile(caminho_arquivo):
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            texto_pre = arquivo.read()
            # Passo 1: Normalizar o texto
            texto_normalizado = normalizar_texto(texto_pre)

            # Passo 2: Remover caracteres especiais
            texto_sem_caracteres_especiais = remover_caracteres_especiais(texto_normalizado)

            # Passo 3: Tokenizar o texto
            tokens = tokenizar_texto(texto_sem_caracteres_especiais)

            # Passo 4: Lematizar os tokens
            tokens_lematizados = lematizar_tokens(tokens)

            # Salvar o texto preprocessado em um arquivo
            caminho_saida = os.path.join(caminho_saida_pasta, "preprocessado_" + arquivo_nome)
            with open(caminho_saida, 'w', encoding='utf-8') as arquivo_saida:
                arquivo_saida.write(" ".join(tokens_lematizados))
            print(f"\nTexto preprocessado salvo em: {caminho_saida}")


O recurso 'punkt' já está disponível.
Baixando o recurso 'stopwords'...

Texto preprocessado salvo em: data/txts_processados\preprocessado_A Cartomante (Machado de Assis).txt


[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\arturevs\Projeto-PLN\.venv\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Texto preprocessado salvo em: data/txts_processados\preprocessado_Dom Casmurro (Machado de Assis).txt

Texto preprocessado salvo em: data/txts_processados\preprocessado_O Cortiço (Aluisio de Azevedo).txt

Texto preprocessado salvo em: data/txts_processados\preprocessado_O Mulato (Aluisio Azevedo).txt
